# Amazon Fine Food Reviews Analysis

Data Source :  https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consist of reviews of fine foods from Amazon.

Number of reviews:568,454
Number of users: 256,059
Number of products:74,258
TimeSpan: Oct 1999 - oct 2012

Number of Attributes/Columns in data: 10

Attribute Information:

    1.Id
    2.ProductId - unique identifier for the product
    3.UserId - unqiue identifier for the user
    4.ProfileName
    5.HelpfulnessNumerator - number of users who found the review helpful
    6.HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
    7.Score - rating between 1 and 5
    8.Time - timestamp for the review
    9.Summary - brief summary of the review
    10.Text - text of the review



Objective:

Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).


[Q] How to determine if a review is positive or negative?

[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

# 1.0 Loading the data

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, con) 
def partition(x):
    if x < 3:
        return 0
    return 1
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


# 1.1 Exploratory Data Analysis


# 1.1.2 Data Cleaning:Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data. Following is an example:


In [3]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that

ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)

ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.


In [4]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [5]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [6]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

Observation:- It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [7]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [8]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [9]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

# 1.1.3 Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

    Begin by removing the html tags
    Remove any punctuations or limited set of special characters like , or . or # etc.
    Check if the word is made up of english letters and is not alpha-numeric
    Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
    Convert the word to lowercase
    Remove Stopwords
    Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)

After which we collect the words used to describe positive and negative reviews


In [10]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [11]:

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'you', 'can', 'further', 'over', 't', 'her', "you'll", 'yours', 'having', 'when', 'more', 'then', 'below', 'same', "mightn't", 'should', 'any', 'being', 'mustn', 'haven', 'itself', 'our', 'after', 'ourselves', 'why', 'are', 'and', 'so', 'doesn', 'won', 'myself', 'by', 'was', "haven't", "mustn't", 'but', 'does', "she's", 'their', 'here', "doesn't", 'or', 'herself', 'all', 'just', 'isn', 'own', 'the', 'as', 'his', 'which', 'in', "hasn't", 'an', 'am', 'will', 'how', 'some', 'd', "you've", "isn't", 'they', 'a', "wasn't", 'we', 'above', "you're", 'be', 'for', 'of', 'while', "don't", 'doing', 'aren', "couldn't", 'hasn', 'mightn', "you'd", 'to', 'ma', 'my', 'each', 'up', 'once', 'what', 'these', 'very', 'before', "needn't", 'has', 'him', 'because', 'until', 'through', 'theirs', 'll', 's', 'where', 'most', 'if', 'again', 'hers', 'weren', 'have', "won't", 'didn', 'been', 'she', 'don', 'your', 'did', 'shan', 'who', 'wasn', 'yourself', "it's", 'wouldn', 'there', 'yourselves', 'i', 'at', 'into', 

In [12]:
if not os.path.isfile('final.sqlite'):
    final_string = []
    all_positive_words = []
    all_negative_words = []
    for i,sent in enumerate(tqdm(final['Text'].values)):
        filtered_sentence = []
        sent = cleanhtml(sent)
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                    if(cleaned_words.lower() not in stop):
                        s = (sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                        if (final['Score'].values)[i] == 1: 
                            all_positive_words.append(s) #list of all words used to describe positive reviews
                        if(final['Score'].values)[i] == 0:
                            all_negative_words.append(s)
        str1 = b" ".join(filtered_sentence)
        final_string.append(str1)
    #Storing data into .sqlite file
    final["CleanedText"] = final_string
    final['CleanedText']=final['CleanedText'].str.decode("utf-8")
    conn = sqlite3.connect('final.sqlite')
    c=conn.cursor()
    conn.text_factory = str
    final.to_sql('Reviews', conn,  schema=None, if_exists='replace', \
                 index=True, index_label=None, chunksize=None, dtype=None)
    conn.close()
    
    with open('positive_words.pkl', 'wb') as f:
        pickle.dump(all_positive_words, f)
    with open('negitive_words.pkl', 'wb') as f:
        pickle.dump(all_negative_words, f)
    

In [13]:
if os.path.isfile('final.sqlite'):
    conn = sqlite3.connect('final.sqlite')
    final = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, conn)
    conn.close()
else:
    print("Please the above cell")

In [14]:
final_df = final.sort_values("Time")

In [15]:
final_df = final_df[0:50000]
train_df = final_df[:40000]
#cv_df = final_df[60000:80000]
test_df = final_df[40000:]

In [16]:
with open('testdf.pkl', 'wb') as f:
        pickle.dump(test_df, f)
with open('traindf.pkl', 'wb') as f:
        pickle.dump(train_df, f)
#with open('cvdf.pkl', 'wb') as f:
 #       pickle.dump(cv_df, f)

In [17]:
#with open('cvdf.pkl', 'rb') as f:
 #       cv_df = pickle.load(f)

# Bag of Words(BoW)

In [18]:
count_vect = CountVectorizer()
trainx = count_vect.fit_transform(train_df['CleanedText'].values)
trainy = train_df["Score"]
#cv_x = count_vect.transform(cv_df['CleanedText'].values)
#cv_y = cv_df["Score"]
testx = count_vect.transform(test_df['CleanedText'].values)
test_y = test_df["Score"]

In [19]:
with open('testbow.pkl', 'wb') as f:
        pickle.dump(testx, f)
with open('trainbow.pkl', 'wb') as f:
        pickle.dump(trainx, f)
#with open('cvbow.pkl', 'wb') as f:
 #       pickle.dump(cv_x, f)

# TF_IDF

In [20]:
tf_idf_vect = TfidfVectorizer()
trainx = tf_idf_vect.fit_transform(train_df['CleanedText'].values)
#trainy = train_df["Score"]
#cv_x = tf_idf_vect.transform(cv_df['CleanedText'].values)
#cv_y = cv_df["Score"]
testx = tf_idf_vect.transform(test_df['CleanedText'].values)
#test_y = test_df["Score"]


In [21]:
word2tfidf = dict(zip(tf_idf_vect.get_feature_names(), tf_idf_vect.idf_))

In [22]:
with open('word2tfidf.pkl', 'wb') as f:
        pickle.dump(word2tfidf, f)

In [23]:
with open('testtfidf.pkl', 'wb') as f:
        pickle.dump(testx, f)
with open('traintfidf.pkl', 'wb') as f:
        pickle.dump(trainx, f)
#with open('cvtfidf.pkl', 'wb') as f:
 #       pickle.dump(cv_x, f)

In [24]:
def list_Sentence(df):
    final = list()
    for i in df["CleanedText"].values:
        l = i.split()
        final.append(l)
    return final

In [25]:
list_train = list_Sentence(train_df)

# Word2Vec

In [26]:
from gensim.models import Word2Vec
model = Word2Vec(list_train)


In [27]:
def avg_w2vec(df):
    vecs = list()
    for i in tqdm(df['CleanedText']):
        length = len(i.split())
        vector = np.zeros(100)
        for word in i.split():
            try:
                x = model[word]
                vector = vector+x
            except:
                pass
                
        vector = vector/length
        vecs.append(vector)
    return vecs

In [28]:
train_avg_w2v = avg_w2vec(train_df)

100%|██████████| 40000/40000 [00:24<00:00, 1600.26it/s]


In [29]:
test_avg_w2v = avg_w2vec(test_df)

100%|██████████| 10000/10000 [00:06<00:00, 1490.34it/s]


In [30]:
#cv_avg_w2v = avg_w2vec(cv_df)

In [31]:
with open('test_avgw2v.pkl', 'wb') as f:
        pickle.dump(test_avg_w2v, f)
with open('train_avgw2v.pkl', 'wb') as f:
        pickle.dump(train_avg_w2v, f)
#with open('cv_avgw2v.pkl', 'wb') as f:
 #       pickle.dump(cv_avg_w2v, f)

In [34]:
testx.shape

(10000, 24413)